### Libraries

In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
pd.set_option('display.max_columns', 100)
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import LabelEncoder
from itertools import combinations

### Read data and perform basic preprocessing

In [2]:
def process_data(DATA_DIR):
    
    train = pd.read_csv(DATA_DIR+"train_Df64byy.csv")
    test = pd.read_csv(DATA_DIR+"test_YCcRUnU.csv")
    
    #Removes train rows which has Region_Code not present in test set
    test_region_list=test['Region_Code'].tolist()
    train=train[train['Region_Code'].isin(test_region_list)]
    
    train['train_or_test']='train'
    test['train_or_test']='test'
    df=pd.concat([train,test])
    
    df['Holding_Policy_Duration']=(df['Holding_Policy_Duration'].replace(['14+'],[15])).astype(float)
    
    le = LabelEncoder()
    for col in ['City_Code','Accomodation_Type','Reco_Insurance_Type','Health Indicator','Is_Spouse']:
        df[col]=  df[col].astype('str')
        df[col]= le.fit_transform(df[col])
        

    
    return train,test,df

### Feature Engineering

In [3]:
def frequency_encoding(column_name,output_column_name,df):
    fe_pol = (df.groupby(column_name).size()) / len(df)
    df[output_column_name] = df[column_name].apply(lambda x : fe_pol[x])

In [4]:
def frequency_encoding(column_name,output_column_name,df):
    fe_pol = (df.groupby(column_name).size()) / len(df)
    df[output_column_name] = df[column_name].apply(lambda x : fe_pol[x])


def feature_engineering(df):
    le = LabelEncoder()
    
     #Interaction Feature (Combining 2 categorical features and performing frequency encoding)
        
    cat_features=[]
    le_features=[]
    columns=['City_Code','Accomodation_Type','Reco_Insurance_Type','Health Indicator','Is_Spouse','Region_Code','Holding_Policy_Type','Reco_Policy_Cat']

    comb = combinations(columns, 2) 

    for i in list(comb):  
        df[f'{i[0]}_{i[1]}']=df[i[0]].astype(str)+'_'+df[i[1]].astype(str)
        df[f'{i[0]}_{i[1]}_le']=le.fit_transform(df[f'{i[0]}_{i[1]}'])
        le_features.append(f'{i[0]}_{i[1]}_le')
        frequency_encoding(f'{i[0]}_{i[1]}',f'{i[0]}_{i[1]}',df)
        cat_features.append(f'{i[0]}_{i[1]}')   
        
    #Frequency Encoding
    
    frequency_encoding('Region_Code','Region_Code_fe',df)
    frequency_encoding('Reco_Policy_Cat','Reco_Policy_Cat_fe',df)
    
    #Deriving characteristics of each city by creating aggregate features
    
    city_aggregate_features = df.groupby(['City_Code']).agg({'Lower_Age': ['mean', 'max', 'min','std'],
                                                     'Reco_Policy_Premium': ['mean', 'max', 'min','std'], 
                                                     'Region_Code': ['nunique','count'], 
                                                     'Accomodation_Type': ['nunique','count'],
                                                     'Reco_Insurance_Type': ['nunique','count'] ,
                                                     'Health Indicator': ['nunique','count'] ,
                                                     'Holding_Policy_Type': ['nunique','count'] ,
                                                     'Reco_Policy_Cat': ['nunique','count'] ,
                                                     })
    city_aggregate_features.columns = ['city_aggregate_features' + '_'.join(c).strip('_') for c in city_aggregate_features.columns]
    df = pd.merge(df, city_aggregate_features, on = ['City_Code'], how='left')

 
    city_region_aggregate_features = df.groupby(['City_Code','Region_Code']).agg({'Lower_Age': ['mean', 'max', 'min','std'],
                                                     'Reco_Policy_Premium': ['mean', 'max', 'min','std'],  
                                                     'Accomodation_Type': ['nunique','count'],
                                                     'Reco_Insurance_Type': ['nunique','count'] ,
                                                     'Health Indicator': ['nunique','count'] ,
                                                     'Holding_Policy_Type': ['nunique','count'] ,
                                                     'Reco_Policy_Cat': ['nunique','count'] ,
                                                     })
    city_region_aggregate_features.columns = ['city_region_aggregate_features' + '_'.join(c).strip('_') for c in city_region_aggregate_features.columns]
    df = pd.merge(df, city_region_aggregate_features, on = ['City_Code','Region_Code'], how='left')

    city_recopolicycat_aggregate_features = df.groupby(['City_Code','Reco_Policy_Cat']).agg({'Lower_Age': ['mean', 'max', 'min','std'],
                                                     'Reco_Policy_Premium': ['mean', 'max', 'min','std'], 
                                                     'Region_Code': ['nunique','count'], 
                                                     'Accomodation_Type': ['nunique','count'],
                                                     'Reco_Insurance_Type': ['nunique','count'] ,
                                                     'Health Indicator': ['nunique','count'] ,
                                                     'Holding_Policy_Type': ['nunique','count'] 
                                                     })
    city_recopolicycat_aggregate_features.columns = ['city_recopolicycat_aggregate_features' + '_'.join(c).strip('_') for c in city_recopolicycat_aggregate_features.columns]
    df = pd.merge(df, city_recopolicycat_aggregate_features, on = ['City_Code','Reco_Policy_Cat'], how='left')
    
    city_regioncoderecopolicycat_aggregate_features = df.groupby(['City_Code','Region_Code_Reco_Policy_Cat']).agg({'Lower_Age': ['mean', 'max', 'min','std'],
                                                     'Reco_Policy_Premium': ['mean', 'max', 'min','std'], 
                                                     'Region_Code': ['nunique','count'], 
                                                     'Accomodation_Type': ['nunique','count'],
                                                     'Reco_Insurance_Type': ['nunique','count'] ,
                                                     'Health Indicator': ['nunique','count'] ,
                                                     'Holding_Policy_Type': ['nunique','count'] ,
                                                     'Reco_Policy_Cat': ['nunique','count'] ,
                                                     })

    city_regioncoderecopolicycat_aggregate_features.columns = ['city_regioncoderecopolicycat_aggregate_features' + '_'.join(c).strip('_') for c in city_regioncoderecopolicycat_aggregate_features.columns]
    df = pd.merge(df, city_regioncoderecopolicycat_aggregate_features, on = ['City_Code','Region_Code_Reco_Policy_Cat'], how='left')
    
    for i in cat_features:
        df[f'city_{i}_max']=df.groupby('City_Code')[i].transform('max')
        df[f'city_{i}_min']=df.groupby('City_Code')[i].transform('min')
        df[f'city_{i}_mean']=df.groupby('City_Code')[i].transform('mean')
        df[f'city_{i}_std']=df.groupby('City_Code')[i].transform('std')

    
        df[f'city_region_{i}_max']=df.groupby(['City_Code','Region_Code'])[i].transform('max')
        df[f'city_region_{i}_min']=df.groupby(['City_Code','Region_Code'])[i].transform('min')
        df[f'city_region_{i}_mean']=df.groupby(['City_Code','Region_Code'])[i].transform('mean')
        df[f'city_region_{i}_std']=df.groupby(['City_Code','Region_Code'])[i].transform('std')

    
        df[f'city_recopolicycat_{i}_max']=df.groupby(['City_Code','Reco_Policy_Cat'])[i].transform('max')
        df[f'city_recopolicycat_{i}_min']=df.groupby(['City_Code','Reco_Policy_Cat'])[i].transform('min')
        df[f'city_recopolicycat_{i}_mean']=df.groupby(['City_Code','Reco_Policy_Cat'])[i].transform('mean')
        df[f'city_recopolicycat_{i}_std']=df.groupby(['City_Code','Reco_Policy_Cat'])[i].transform('std')
        
    
    #features on reco_policy_cat
    
    recopolicycat_aggregate_features = df.groupby(['Reco_Policy_Cat']).agg({'Lower_Age': ['mean', 'max', 'min','std'],
                                                     'Reco_Policy_Premium': ['mean', 'max', 'min','std','sum'],   
                                                     'Region_Code': ['nunique','count'], 
                                                     'Accomodation_Type': ['nunique'],
                                                     'Reco_Insurance_Type': ['nunique'] ,
                                                     'Health Indicator': ['nunique','count'] ,
                                                     'Holding_Policy_Type': ['nunique','count'] ,
                                                     'City_Code': ['nunique','count'] ,
                                                     })
    recopolicycat_aggregate_features.columns = ['recopolicycat_aggregate_features' + '_'.join(c).strip('_') for c in recopolicycat_aggregate_features.columns]
    df = pd.merge(df, recopolicycat_aggregate_features, on = ['Reco_Policy_Cat'], how='left')
        
        

    #features on Holding_Policy_Type 
    
    holdingpolicytype_aggregate_features = df.groupby(['Holding_Policy_Type']).agg({'Lower_Age': ['mean', 'max', 'min','std'],
                                                     'Reco_Policy_Premium': ['mean', 'max', 'min','std'], 
                                                     'Region_Code': ['nunique','count'], 
                                                     'Accomodation_Type': ['nunique','count'],
                                                     'Reco_Insurance_Type': ['nunique','count'] ,
                                                     'Health Indicator': ['nunique','count'] ,
                                                     'City_Code': ['nunique','count'] ,
                                                     })
    holdingpolicytype_aggregate_features.columns = ['holdingpolicytype_aggregate_features' + '_'.join(c).strip('_') for c in holdingpolicytype_aggregate_features.columns]
    df = pd.merge(df, holdingpolicytype_aggregate_features, on = ['Holding_Policy_Type'], how='left')
    
    #Deriving characteristics of Accomodation_Type by creating aggregate features
    
    Accomodation_Type_aggregate_features = df.groupby(['Accomodation_Type']).agg({'Lower_Age': ['mean', 'max', 'min','std'],
                                                     'Reco_Policy_Premium': ['mean', 'max', 'min','std','sum'],   
                                                     'Region_Code': ['nunique','count'], 
                                                     'Reco_Insurance_Type': ['nunique'] ,
                                                     'Health Indicator': ['nunique','count'] ,
                                                     'Holding_Policy_Type': ['nunique','count'] ,
                                                     'City_Code': ['nunique','count'] ,
                                                     })
    Accomodation_Type_aggregate_features.columns = ['Accomodation_Type_aggregate_features' + '_'.join(c).strip('_') for c in Accomodation_Type_aggregate_features.columns]
    df = pd.merge(df, Accomodation_Type_aggregate_features, on = ['Accomodation_Type'], how='left')
    
    #Deriving characteristics of Interaction_features by creating aggregate features (These interaction feature are selected for aggregating based on its feature importance)
    
    Region_CodeReco_Policy_Cat_grpd = df.groupby(['Region_Code_Reco_Policy_Cat']).agg({ 'Reco_Policy_Premium': ['mean', 'max', 'min', 'std']})                                                              
                                                     
    Region_CodeReco_Policy_Cat_grpd.columns = ['grpd_by_Region_Code_Reco_Policy_Cat_' + '_'.join(c).strip('_') for c in Region_CodeReco_Policy_Cat_grpd.columns]
    df = pd.merge(df, Region_CodeReco_Policy_Cat_grpd, on = ['Region_Code_Reco_Policy_Cat'], how='left')


    City_CodeRegion_Code_grpd = df.groupby(['City_Code_Region_Code']).agg({ 'Reco_Policy_Premium': ['mean', 'max', 'min', 'std']})                                                              
                                                     
    City_CodeRegion_Code_grpd.columns = ['grpd_by_City_CodeRegion_Code_' + '_'.join(c).strip('_') for c in City_CodeRegion_Code_grpd.columns]
    df = pd.merge(df, City_CodeRegion_Code_grpd, on = ['City_Code_Region_Code'], how='left')


    City_CodeReco_Policy_Cat_grpd = df.groupby(['City_Code_Reco_Policy_Cat']).agg({ 'Reco_Policy_Premium': ['mean', 'max', 'min', 'std']})                                                              
                                                     
    City_CodeReco_Policy_Cat_grpd.columns = ['grpd_by_City_CodeReco_Policy_Cat_' + '_'.join(c).strip('_') for c in City_CodeReco_Policy_Cat_grpd.columns]
    df = pd.merge(df, City_CodeReco_Policy_Cat_grpd, on = ['City_Code_Reco_Policy_Cat'], how='left')


    Holding_Policy_TypeReco_Policy_Cat_grpd = df.groupby(['Holding_Policy_Type_Reco_Policy_Cat']).agg({ 'Reco_Policy_Premium': ['mean', 'max', 'min', 'std']})                                                              
                                                     
    Holding_Policy_TypeReco_Policy_Cat_grpd.columns = ['grpd_by_Holding_Policy_TypeReco_Policy_Cat_' + '_'.join(c).strip('_') for c in Holding_Policy_TypeReco_Policy_Cat_grpd.columns]
    df = pd.merge(df, Holding_Policy_TypeReco_Policy_Cat_grpd, on = ['Holding_Policy_Type_Reco_Policy_Cat'], how='left')
    
    return df,le_features
    

### Remove unnecessary columns and prepare the train and test data for training

In [5]:
def preparedatafortraining(df,train,test):
    
    train=df.loc[df.train_or_test.isin(['train'])]
    test=df.loc[df.train_or_test.isin(['test'])]
    
    drop_columns={'ID','Response','Upper_Age','train_or_test'}
    
    target=['Response']
    
    x=train.drop(columns=drop_columns,axis=1)
    y=train[target]
    x_test=test.drop(columns=drop_columns,axis=1)
    
    print(x.shape)
    
    return x,y,x_test

### Save Data

In [6]:
def savedata(**DATA_DIR):
    
    train,test,df=process_data("../input/analytics-vidhya-jobathon/")
    df,cat_features=feature_engineering(df)
    x_train,y_train,x_test=preparedatafortraining(df,train,test)
    
    #x_train.to_pickle("x_train_lgbm.pkl")
    #y_train.to_pickle("y_train_lgbm.pkl")
    #x_test.to_pickle("x_test_lgbm.pkl")
    
    return x_train,y_train,x_test,cat_features

### Train CatBoost Model and save the validation and test set prediction for ensembling

In [7]:
def catboost_model():
    
    x,y,x_test,cat_features=savedata()
     
    err = [] 

    oofs = np.zeros(shape=(len(x)))
    preds = np.zeros(shape=(len(x_test)))

    Folds=8

    fold = StratifiedKFold(n_splits=Folds, shuffle=True, random_state=2020)
    i = 1

    for train_index, test_index in fold.split(x, y):
        x_train, x_val = x.iloc[train_index], x.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    
        m =  CatBoostClassifier(n_estimators=10000,random_state=2020,eval_metric='AUC')
    
        m.fit(x_train, y_train,eval_set=[(x_val, y_val)], early_stopping_rounds=30,verbose=100,cat_features=cat_features)
    
        pred_y = m.predict_proba(x_val)[:,1]
        oofs[test_index] = pred_y
        print(i, " err_cat: ", roc_auc_score(y_val,pred_y))
        err.append(roc_auc_score(y_val,pred_y))
        preds+= m.predict_proba(x_test)[:,1]
        i = i + 1
    preds=preds/Folds
    
    print(f"Average StratifiedKFold Score : {sum(err)/Folds} ")
    oof_score = roc_auc_score(y, oofs)
    print(f'\nOOF Auc is : {oof_score}')
    
    oofs=pd.DataFrame(oofs,columns=['catboostoof'])
    preds=pd.DataFrame(preds,columns=['catboostpred'])
    
    oofs.to_csv('catboostoof.csv',index=False)
    preds.to_csv('catboostpred.csv',index=False)

In [8]:
catboost_model()

(48752, 552)
Learning rate set to 0.029412
0:	test: 0.7636705	best: 0.7636705 (0)	total: 336ms	remaining: 55m 56s
100:	test: 0.8133179	best: 0.8133179 (100)	total: 25.7s	remaining: 42m 1s
200:	test: 0.8176711	best: 0.8176711 (200)	total: 50.5s	remaining: 41m 4s
300:	test: 0.8191514	best: 0.8191629 (299)	total: 1m 14s	remaining: 40m 14s
400:	test: 0.8204221	best: 0.8204221 (400)	total: 1m 39s	remaining: 39m 34s
500:	test: 0.8214007	best: 0.8214007 (500)	total: 2m 2s	remaining: 38m 49s
600:	test: 0.8223918	best: 0.8224123 (597)	total: 2m 27s	remaining: 38m 21s
700:	test: 0.8231721	best: 0.8231721 (700)	total: 2m 51s	remaining: 37m 49s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.823356621
bestIteration = 719

Shrink model to first 720 iterations.
1  err_cat:  0.8233566210205541
Learning rate set to 0.029412
0:	test: 0.7472616	best: 0.7472616 (0)	total: 274ms	remaining: 45m 36s
100:	test: 0.7987846	best: 0.7987846 (100)	total: 25.6s	remaining: 41m 52s
200:	test: 0.8